In [1]:
import pandas as pd 
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import xgboost


In [2]:
stroke_df = pd.read_csv('./healthcare-dataset-stroke-data.csv')
pd.set_option("display.max_rows", None)
stroke_df.head()
stroke_df.dtypes
stroke_df['bmi'] = stroke_df['bmi'].fillna(0)

In [3]:
stroke_df.isna().sum()


id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [4]:
stroke_df.nunique()


id                   5110
gender                  3
age                   104
hypertension            2
heart_disease           2
ever_married            2
work_type               5
Residence_type          2
avg_glucose_level    3979
bmi                   419
smoking_status          4
stroke                  2
dtype: int64

In [5]:
stroke_df.drop(columns='id', inplace=True)
stroke_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,0.0,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [6]:
df_dummies = pd.get_dummies(stroke_df[['gender','ever_married', 'work_type','Residence_type', 'smoking_status']])
stroke_df_copy = stroke_df[['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'stroke']]
merged_data = pd.concat([stroke_df_copy, df_dummies], axis= 1)
merged_data.head()
X = merged_data.drop('stroke', axis=1).values
y = merged_data['stroke'].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [8]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
len(X_train[0])

21

In [11]:
nn = tf.keras.models.Sequential()
number_of_input_features = len(X_train[0])
hidden_neuron1 = 100 
hidden_neuron2 = 50 
hidden_neuron3 = 30 

nn.add(tf.keras.layers.Dense(units = hidden_neuron1, activation='relu', input_dim = number_of_input_features))
nn.add(tf.keras.layers.Dense(units = hidden_neuron2, activation='sigmoid'))
nn.add(tf.keras.layers.Dense(units = hidden_neuron3, activation='sigmoid'))
nn.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               2200      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 30)                1530      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 8,811
Trainable params: 8,811
Non-trainable params: 0
_________________________________________________________________


In [12]:
nn.compile(loss= 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [13]:
nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
120/120 [==============================] - 2s 3ms/step - loss: 0.1937 - accuracy: 0.9559
Epoch 2/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1648 - accuracy: 0.9559
Epoch 3/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1514 - accuracy: 0.9559
Epoch 4/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1463 - accuracy: 0.9559
Epoch 5/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1415 - accuracy: 0.9559
Epoch 6/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1398 - accuracy: 0.9559
Epoch 7/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1376 - accuracy: 0.9559
Epoch 8/100
120/120 [==============================] - 0s 4ms/step - loss: 0.1368 - accuracy: 0.9559
Epoch 9/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1354 - accuracy: 0.9559
Epoch 10/100
120/120 [==============================] - 0s 3ms/step - loss: 0.1337 - accura

In [14]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose =2)
print(f"Model_loss: {model_loss}, Model_accuracy: {model_accuracy}")

40/40 - 1s - loss: 0.2662 - accuracy: 0.9249 - 643ms/epoch - 16ms/step
Model_loss: 0.26620104908943176, Model_accuracy: 0.9248826503753662


In [15]:
xgb_model = xgboost.XGBClassifier()
xgb_model = xgb_model.fit(X_train_scaled, y_train)
predictions = xgb_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predictions)
accuracy

0.9327073552425665

In [17]:
forest_model = RandomForestClassifier(n_estimators= 398, random_state= 42)
forest_model = forest_model.fit(X_train_scaled, y_train)
predictions = forest_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, predictions)
accuracy 

0.9381846635367762